In [1]:
################################################################################################################################
# git clone git@github.com:Diego999/pyGAT.git
################################################################################################################################
# 环境搭建：
# conda create -n GNN_GAT
# conda activate GNN_GAT

# conda install python=3.8

# https://pytorch.org/get-started/previous-versions/
# conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.7 -c pytorch -c nvidia
################################################################################################################################
# conda install ipykernel
# conda install platformdirs
# pip3 install ipywidgets
# pip3 install --upgrade jupyter_core jupyter_client

# python -m ipykernel install --user --name GNN_GAT
################################################################################################################################


################################################################################################################################
# pip install scipy

# sudo apt install graphviz
# pip install graphviz
################################################################################################################################

In [2]:
import argparse
import glob
import numpy as np
import os
import random
import scipy.sparse as sp
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from __future__ import division
from __future__ import print_function

In [3]:
class GraphAttentionLayer(nn.Module):


    def __init__(self, in_features, out_features, dropout, alpha, concat=True):

        super(GraphAttentionLayer, self).__init__()

        self.in_features = in_features  # I
        self.out_features = out_features  # O
        self.dropout = dropout
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))  # I*O
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1)))  # 2O*1
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)


    def forward(self, adj, h):  #

        Wh = torch.mm(h, self.W)  # h(N*I) * W(I*O) = Wh(N*O)
        e = self._prepare_attentional_mechanism_input(Wh)  # 全关联系数矩阵

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj>0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, Wh)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime


    def _prepare_attentional_mechanism_input(self, Wh):

        # Wh(N*O) * a(1/2)(O*1) = Wh(1/2)(N*1)，每个节点的加与被加关联值
        Wh1 = torch.matmul(Wh, self.a[:self.out_features, :])  # https://blog.csdn.net/didi_ya/article/details/121158666
        Wh2 = torch.matmul(Wh, self.a[self.out_features:, :])  # https://blog.csdn.net/weixin_48780159/article/details/119032864

        # broadcast add
        e = Wh1 + Wh2.T  # e(N*N)

        return self.leakyrelu(e)


    def __repr__(self):
        return self.__class__.__name__ + '(' + str(self.in_features) + '->' + str(self.out_features) + ')'

In [4]:
class GAT(nn.Module):


    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):

        super(GAT, self).__init__()

        self.dropout = dropout

        # 多头
        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)  # 

        # 两层
        self.out_att = GraphAttentionLayer(nhid*nheads, nclass, dropout=dropout, alpha=alpha, concat=False)


    def forward(self, adj, x):

        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(adj, x) for att in self.attentions], dim=1)

        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(adj, x))

        return F.log_softmax(x, dim=1)

In [5]:
def encode_onehot(labels):

    # The classes must be sorted before encoding to enable static class encoding.
    # In other words, make sure the first class always maps to index 0.
    classes = sorted(list(set(labels)))
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)

    return labels_onehot


def load_data(path="./cora/", dataset="cora"):

    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T>adj) - adj.multiply(adj.T>adj)

    features = normalize_features(features)
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    adj = torch.FloatTensor(np.array(adj.todense()))
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test


def normalize_adj(mx):

    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)

    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)


def normalize_features(mx):

    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)

    return mx


def accuracy(output, labels):

    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()

    return correct / len(labels)

In [6]:
# seed
random.seed(72)
np.random.seed(72)
torch.manual_seed(72)
if torch.cuda.is_available():
    torch.cuda.manual_seed(72)


# load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()


# model and optimizer
model = GAT(nfeat=features.shape[1], 
            nhid=8, 
            nclass=int(labels.max()) + 1, 
            dropout=0.6, 
            nheads=8, 
            alpha=0.2)
optimizer = optim.Adam(model.parameters(), 
                       lr=0.005, 
                       weight_decay=5e-4)


# cuda
if torch.cuda.is_available():
    adj = adj.cuda()
    features = features.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()
    model.cuda()


# 
adj, features, labels = Variable(adj), Variable(features), Variable(labels)

/home/zhfc/anaconda3/envs/GNN_GAT/lib/python3.8/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541702/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Loading cora dataset...


In [7]:
def train(epoch):

    t = time.time()

    # train
    model.train()
    optimizer.zero_grad()

    output = model(adj, features)

    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    # evaluate
    # deactivates dropout during validation run.
    model.eval()
    output = model(adj, features)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])

    print('Epoch: {:04d}'.format(epoch+1), 
          'loss_train: {:.4f}'.format(loss_train.data.item()), 
          'acc_train: {:.4f}'.format(acc_train.data.item()), 
          'loss_val: {:.4f}'.format(loss_val.data.item()), 
          'acc_val: {:.4f}'.format(acc_val.data.item()), 
          'time: {:.4f}s'.format(time.time() - t))

    return loss_val.data.item()

In [8]:
def compute_test():

    model.eval()
    output = model(adj, features)

    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])

    print("Test set results:", 
          "loss= {:.4f}".format(loss_test.data.item()),
          "accuracy= {:.4f}".format(acc_test.data.item()))

In [9]:
# train model
t_total = time.time()

loss_values = []
bad_counter = 0
best = 10000 + 1
best_epoch = 0


for epoch in range(10000):

    loss_values.append(train(epoch))

    torch.save(model.state_dict(), '{}.pkl'.format(epoch))

    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == 100:
        break

    files = glob.glob('*.pkl')
    for file in files:
        epoch_nb = int(file.split('.')[0])
        if epoch_nb < best_epoch:
            os.remove(file)


files = glob.glob('*.pkl')
for file in files:
    epoch_nb = int(file.split('.')[0])
    if epoch_nb > best_epoch:
        os.remove(file)


print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Restore best model
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))


# Testing
compute_test()

Epoch: 0001 loss_train: 1.9486 acc_train: 0.1714 loss_val: 1.9411 acc_val: 0.2533 time: 0.4625s
Epoch: 0002 loss_train: 1.9442 acc_train: 0.2071 loss_val: 1.9323 acc_val: 0.4800 time: 0.3920s
Epoch: 0003 loss_train: 1.9291 acc_train: 0.3357 loss_val: 1.9231 acc_val: 0.5900 time: 0.3973s
Epoch: 0004 loss_train: 1.9233 acc_train: 0.3429 loss_val: 1.9140 acc_val: 0.6267 time: 0.4066s
Epoch: 0005 loss_train: 1.9044 acc_train: 0.4714 loss_val: 1.9050 acc_val: 0.6333 time: 0.4121s
Epoch: 0006 loss_train: 1.8991 acc_train: 0.5286 loss_val: 1.8959 acc_val: 0.6533 time: 0.4318s
Epoch: 0007 loss_train: 1.8786 acc_train: 0.5286 loss_val: 1.8865 acc_val: 0.6500 time: 0.4275s
Epoch: 0008 loss_train: 1.8726 acc_train: 0.5643 loss_val: 1.8770 acc_val: 0.6433 time: 0.4054s
Epoch: 0009 loss_train: 1.8614 acc_train: 0.5500 loss_val: 1.8675 acc_val: 0.6433 time: 0.4609s
Epoch: 0010 loss_train: 1.8347 acc_train: 0.6214 loss_val: 1.8576 acc_val: 0.6500 time: 0.3901s
Epoch: 0011 loss_train: 1.8332 acc_train